In [1]:
# get openai api key from openai_config.json file
import json
with open('../openai_config.json') as f:
    openai_api_key = json.load(f)['openai_api_key']

In [2]:
%%capture

%env OPENAI_API_KEY = {openai_api_key}

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")
llm.invoke("Hello, how are you?").content

"Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?"

In [58]:
from langchain.document_loaders import CSVLoader
input_loader_kor = CSVLoader(r"../../guidance_for_environmental_impact_factor_mapping_on_aws/assets/input/coupang_product_names_groceries_v3_kor.csv", encoding='utf-8-sig')
input_doc_kor = input_loader_kor.load()

input_doc_kor[100].page_content

'품목명: 아이시스 8.0 생수, 200ml, 40개'

In [59]:
from langchain_core.prompts import PromptTemplate

prompt = """너는 식료품을 HS 코드로 분류하는 전과정영향평가(Life Cycle Analysis, LCA) 전문가야.

나는 환경 확장 투입 산출(Environmentally Extended Input Output, EEIO) 환경 영향 계수(Environmental Impact Factors, EIF)를 기반으로 한 식료품의 전과정영향평가를 수행하고 싶어. 나는 식료품의 재료 및 제조 단계와 관련된 환경적 영향에 관심 있어.

나는 주어진 물품에 가장 적합한 HS 코드와 항목명을 알고 싶어. 선택에 대한 근거를 함께 설명해야 해.
출력 형식은 JSON으로, 키는 "가장_적합한_HS_코드", "가장_적합한_HS_항목명", "판단_근거"야.

물품:
{product_name}

다음 중 제공된 물품과 가장 적합한 HS 코드는 무엇이지?

주어진 정보를 최대한 활용해. 정보가 제한되어 있다고 말하거나 추가 정보를 요청하지 마.
가장 적합한 코드와 항목명 선택해야 해. 선택에 대한 근거를 반드시 포함해야 해.
주어진 지침이나 정보를 반복하지 마.
전과정영향평가에 필요한 정보가 충분하지 않다고 말하지 마.
다른 말 없이 JSON 출력으로만 응답해."""
prompt = PromptTemplate.from_template(prompt)

In [ ]:
# Initialize results storage
results_kor = []

# Process each row using the graph
for doc in input_doc_kor[:1]:
    # # Extract CommodityDescription from the page_content using regex
    # match = re.search(r"CommodityDescription: (.+)", doc.page_content)
    # activity = match.group(1).strip() if match else None  # Extract the description

    activity = doc.page_content # Extract the Product Name

    if activity:
        msg = prompt.invoke({"product_name": activity})
        result = json.loads(llm.invoke(msg).content.strip("```").lstrip("json").replace("\n", "").replace(" ", ""))
        result['상품_이름'] = activity
        result["가장_적합한_HS_코드"] = result.get("가장_적합한_HS_코드").replace(".", "").rjust(6, "0")
        
        # Extract context and answer from the state after processing
        results_kor.append(result)

# Print the results for debugging
print("Final results:", results_kor[0])

Final results: [{'가장_적합한_HS_코드': '190490', '가장_적합한_HS_항목명': '기타조리된식사류', '판단_근거': '오뚜기흰밥은이미조리된밥으로,HS코드1904.90은곡물이나곡물제품을사용하여조리된즉석식품에해당한다.210g의개별포장된즉석흰밥형태로판매되므로이코드는해당제품에적합하다.', '상품_이름': '품목명: 오뚜기 흰밥, 210g, 24개'}]


In [ ]:
import csv
to_csv = results_kor
keys = to_csv[0].keys()

with open("coupang_openai_gt_all", 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(to_csv)